In [1]:
!pip install tensorlayer

# Connecting to Gdrive

In [2]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
gdrive = GoogleDrive(gauth)

# PyDrive reference:
# https://gsuitedevs.github.io/PyDrive/docs/build/html/index.html

# 2. Create & upload a file text file.
uploaded = gdrive.CreateFile({'title': 'Sample upload.txt'})
uploaded.SetContentString('Sample upload file content')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

# 3. Load a file by ID and print its contents.
downloaded = gdrive.CreateFile({'id': uploaded.get('id')})
print('Downloaded content "{}"'.format(downloaded.GetContentString()))

Uploaded file with ID 1jm_CZnJshY8B_lSjlfuT8l_OzMfKIudr
Downloaded content "Sample upload file content"


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# Imports

In [0]:
import numpy as np
import pickle

import tensorflow as tf
import tensorlayer as tl

from tqdm import tqdm
from sklearn.utils import shuffle
from tensorlayer.layers import DenseLayer, EmbeddingInputlayer, Seq2Seq, retrieve_seq_length_op2

sess_config = tf.ConfigProto(allow_soft_placement=True, log_device_placement=False)

In [0]:
drive_path = 'gdrive/My Drive/masters/kpis/chatbot/data/'

# Initial Setup

In [0]:
def load_data(PATH=drive_path):
    # read data control dictionaries
    try:
        with open(PATH + 'metadata.pkl', 'rb') as f:
            metadata = pickle.load(f)
    except:
        metadata = None
    # read numpy arrays
    idx_q = np.load(PATH + 'idx_q.npy')
    idx_a = np.load(PATH + 'idx_a.npy')
    return metadata, idx_q, idx_a
  
'''
 split data into train (70%), test (15%) and valid(15%)
    return tuple( (trainX, trainY), (testX,testY), (validX,validY) )

'''
def split_dataset(x, y, ratio = [0.7, 0.15, 0.15] ):
    # number of examples
    data_len = len(x)
    lens = [ int(data_len*item) for item in ratio ]

    trainX, trainY = x[:lens[0]], y[:lens[0]]
    testX, testY = x[lens[0]:lens[0]+lens[1]], y[lens[0]:lens[0]+lens[1]]
    validX, validY = x[-lens[-1]:], y[-lens[-1]:]

    return (trainX,trainY), (testX,testY), (validX,validY)

In [0]:
def initial_setup():
    metadata, idx_q, idx_a = load_data()
    (trainX, trainY), (testX, testY), (validX, validY) = split_dataset(idx_q, idx_a)
    trainX = tl.prepro.remove_pad_sequences(trainX.tolist())
    trainY = tl.prepro.remove_pad_sequences(trainY.tolist())
    testX = tl.prepro.remove_pad_sequences(testX.tolist())
    testY = tl.prepro.remove_pad_sequences(testY.tolist())
    validX = tl.prepro.remove_pad_sequences(validX.tolist())
    validY = tl.prepro.remove_pad_sequences(validY.tolist())
    return metadata, trainX, trainY, testX, testY, validX, validY

# Creating the LSTM model 

In [0]:
"""
Creates the LSTM Model
"""
def create_model(encode_seqs, decode_seqs, src_vocab_size, emb_dim, is_train=True, reuse=False):
    with tf.variable_scope("model", reuse=reuse):
        # for chatbot, you can use the same embedding layer,
        # for translation, you may want to use 2 seperated embedding layers
        with tf.variable_scope("embedding") as vs:
            net_encode = EmbeddingInputlayer(
                inputs = encode_seqs,
                vocabulary_size = src_vocab_size,
                embedding_size = emb_dim,
                name = 'seq_embedding')
            vs.reuse_variables()
            net_decode = EmbeddingInputlayer(
                inputs = decode_seqs,
                vocabulary_size = src_vocab_size,
                embedding_size = emb_dim,
                name = 'seq_embedding')
            
        net_rnn = Seq2Seq(net_encode, net_decode,
                cell_fn = tf.nn.rnn_cell.LSTMCell,
                n_hidden = emb_dim,
                initializer = tf.random_uniform_initializer(-0.1, 0.1),
                encode_sequence_length = retrieve_seq_length_op2(encode_seqs),
                decode_sequence_length = retrieve_seq_length_op2(decode_seqs),
                initial_state_encode = None,
                dropout = (0.5 if is_train else None),
                n_layer = 3,
                return_seq_2d = True,
                name = 'seq2seq')

        net_out = DenseLayer(net_rnn, n_units=src_vocab_size, act=tf.identity, name='output')
    return net_out, net_rnn

**CREATING A FILE IN GOOGLE DRIVE TO SAVE MODEL**

In [0]:
def does_file_exists_in_gdrive(name='', folder_id = "root", is_folder= True):
  file_list = gdrive.ListFile({'q':"'{0}' in parents and trashed=false".format(folder_id), 'maxResults': 10}).GetList()
  file_exists = False
  id =None

  for file1 in file_list:
    if(file1['title'] == name):
      file_exists = True
      id = file1['id']
      break
  if not file_exists and is_folder:
    folder_metadata = {'title' : name, 'mimeType' : 'application/vnd.google-apps.folder'}
    folder = gdrive.CreateFile(folder_metadata)
    folder.Upload()
    id = folder['id']
  return (id, file_exists)

In [0]:
id,_ = does_file_exists_in_gdrive(name='Chatbot_model', folder_id="root", is_folder=True)
file_id, doesFileExist = does_file_exists_in_gdrive(name='model.npz', folder_id=id, is_folder= False)

if doesFileExist:
  model_file = gdrive.CreateFile({"parents": [{"kind": "drive#fileLink", "id": id}], "id": file_id})
else:
  model_file = gdrive.CreateFile({"parents": [{"kind": "drive#fileLink", "id": id}]})

In [31]:
_

True

# Training utility 

In [0]:
def train(batch_size, num_epochs, learning_rate, inference_mode):

    metadata, trainX, trainY, testX, testY, validX, validY = initial_setup()

    # Parameters
    src_len = len(trainX)
    tgt_len = len(trainY)

    assert src_len == tgt_len
    print(src_len)
    
    n_step = src_len // batch_size
    src_vocab_size = len(metadata['idx2w']) # 8002 (0~8001)
    emb_dim = 1024

    word2idx = metadata['w2idx']   # dict  word 2 index
    idx2word = metadata['idx2w']   # list index 2 word

    unk_id = word2idx['unk']   # 1
    pad_id = word2idx['_']     # 0

    start_id = src_vocab_size  # 8002
    end_id = src_vocab_size + 1  # 8003

    word2idx.update({'start_id': start_id})
    word2idx.update({'end_id': end_id})
    idx2word = idx2word + ['start_id', 'end_id']

    src_vocab_size = tgt_vocab_size = src_vocab_size + 2

    """ A data for Seq2Seq should look like this:
    input_seqs : ['how', 'are', 'you', '<PAD_ID'>]
    decode_seqs : ['<START_ID>', 'I', 'am', 'fine', '<PAD_ID'>]
    target_seqs : ['I', 'am', 'fine', '<END_ID>', '<PAD_ID'>]
    target_mask : [1, 1, 1, 1, 0]
    """
    # Preprocessing
    target_seqs = tl.prepro.sequences_add_end_id([trainY[10]], end_id=end_id)[0]
    decode_seqs = tl.prepro.sequences_add_start_id([trainY[10]], start_id=start_id, remove_last=False)[0]
    target_mask = tl.prepro.sequences_get_mask([target_seqs])[0]
    if not inference_mode:
        print("encode_seqs", [idx2word[id] for id in trainX[10]])
        print("target_seqs", [idx2word[id] for id in target_seqs])
        print("decode_seqs", [idx2word[id] for id in decode_seqs])
        print("target_mask", target_mask)
        print(len(target_seqs), len(decode_seqs), len(target_mask))

    # Init Session
    tf.reset_default_graph()
    sess = tf.Session(config=sess_config)

    # Training Data Placeholders
    encode_seqs = tf.placeholder(dtype=tf.int64, shape=[batch_size, None], name="encode_seqs")
    decode_seqs = tf.placeholder(dtype=tf.int64, shape=[batch_size, None], name="decode_seqs")
    target_seqs = tf.placeholder(dtype=tf.int64, shape=[batch_size, None], name="target_seqs")
    target_mask = tf.placeholder(dtype=tf.int64, shape=[batch_size, None], name="target_mask") 

    net_out, _ = create_model(encode_seqs, decode_seqs, src_vocab_size, emb_dim, is_train=True, reuse=False)
    net_out.print_params(False)

    # Inference Data Placeholders
    encode_seqs2 = tf.placeholder(dtype=tf.int64, shape=[1, None], name="encode_seqs")
    decode_seqs2 = tf.placeholder(dtype=tf.int64, shape=[1, None], name="decode_seqs")

    net, net_rnn = create_model(encode_seqs2, decode_seqs2, src_vocab_size, emb_dim, is_train=False, reuse=True)
    y = tf.nn.softmax(net.outputs)

    # Loss Function
    loss = tl.cost.cross_entropy_seq_with_mask(logits=net_out.outputs, target_seqs=target_seqs, 
                                                input_mask=target_mask, return_details=False, name='cost')

    # Optimizer
    train_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

    # Init Vars
    sess.run(tf.global_variables_initializer())

    # Load Model
    tl.files.load_and_assign_npz(sess=sess, name='gdrive/My Drive/Chatbot_model/model.npz', network=net_out)
    tl.files.load_and_assign_npz(sess=sess, name='model.npz', network=net)

    """
    Inference using pre-trained model
    """
    def inference(seed):
        seed_id = [word2idx.get(w, unk_id) for w in seed.split(" ")]
        
        # Encode and get state
        state = sess.run(net_rnn.final_state_encode,
                        {encode_seqs2: [seed_id]})
        # Decode, feed start_id and get first word [https://github.com/zsdonghao/tensorlayer/blob/master/example/tutorial_ptb_lstm_state_is_tuple.py]
        o, state = sess.run([y, net_rnn.final_state_decode],
                        {net_rnn.initial_state_decode: state,
                        decode_seqs2: [[start_id]]})
        w_id = tl.nlp.sample_top(o[0], top_k=3)
        w = idx2word[w_id]
        # Decode and feed state iteratively
        sentence = [w]
        for _ in range(30): # max sentence length
            o, state = sess.run([y, net_rnn.final_state_decode],
                            {net_rnn.initial_state_decode: state,
                            decode_seqs2: [[w_id]]})
            w_id = tl.nlp.sample_top(o[0], top_k=2)
            w = idx2word[w_id]
            if w_id == end_id:
                break
            sentence = sentence + [w]
        return sentence

    if inference_mode:
        print('Inference Mode')
        print('--------------')
        while True:
            input_seq = input('Enter Query: ')
            sentence = inference(input_seq)
            print(" >", ' '.join(sentence))
    else:
        seeds = ["you want to turn twitter followers into blog readers",
                 "does the dog also have polio", 
                 "wow, this is such a gorgeous photo. what did you use to take it",
                 "i bet these guys that are playing d for philly feel so fresh"]
        for epoch in range(num_epochs):
            trainX, trainY = shuffle(trainX, trainY, random_state=0)
            total_loss, n_iter = 0, 0
            for X, Y in tqdm(tl.iterate.minibatches(inputs=trainX, targets=trainY, batch_size=batch_size, shuffle=False), 
                            total=n_step, desc='Epoch[{}/{}]'.format(epoch + 1, num_epochs), leave=False):

                X = tl.prepro.pad_sequences(X)
                _target_seqs = tl.prepro.sequences_add_end_id(Y, end_id=end_id)
                _target_seqs = tl.prepro.pad_sequences(_target_seqs)
                _decode_seqs = tl.prepro.sequences_add_start_id(Y, start_id=start_id, remove_last=False)
                _decode_seqs = tl.prepro.pad_sequences(_decode_seqs)
                _target_mask = tl.prepro.sequences_get_mask(_target_seqs)
                ## Uncomment to view the data here
                # for i in range(len(X)):
                #     print(i, [idx2word[id] for id in X[i]])
                #     print(i, [idx2word[id] for id in Y[i]])
                #     print(i, [idx2word[id] for id in _target_seqs[i]])
                #     print(i, [idx2word[id] for id in _decode_seqs[i]])
                #     print(i, _target_mask[i])
                #     print(len(_target_seqs[i]), len(_decode_seqs[i]), len(_target_mask[i]))
                _, loss_iter = sess.run([train_op, loss], {encode_seqs: X, decode_seqs: _decode_seqs,
                                target_seqs: _target_seqs, target_mask: _target_mask})
                total_loss += loss_iter
                n_iter += 1
            print('n_iter is {0}'.format(n_iter))
            # printing average loss after every epoch
            print('Epoch [{}/{}]: loss {:.4f}'.format(epoch + 1, num_epochs, total_loss / n_iter))
            
            # inference after every epoch
            for seed in seeds:
                print("Query >", seed)
                for _ in range(5):
                    sentence = inference(seed)
                    print(" >", ' '.join(sentence))
            
            # saving the model
            tl.files.save_npz(net.all_params, name='model.npz', sess=sess)
            model_file.SetContentFile('model.npz')
            model_file.Upload()
    # session cleanup
    sess.close()

# Training

In [0]:
train(batch_size=32, num_epochs = 50, learning_rate = 0.004, inference_mode= False)

91295
encode_seqs ['wednesday', 'and', 'thursday', 'in', 'seattle', 's', 'big', 'conference', '']
target_seqs ['wish', 'but', 'cant', 'travel', 'enjoy', 'end_id']
decode_seqs ['start_id', 'wish', 'but', 'cant', 'travel', 'enjoy']
target_mask [1 1 1 1 1 1]
6 6 6
[TL] EmbeddingInputlayer model/embedding/seq_embedding: (8004, 1024)
[TL] EmbeddingInputlayer model/embedding/seq_embedding: (8004, 1024)
[TL] [*] Seq2Seq model/seq2seq: n_hidden: 1024 cell_fn: LSTMCell dropout: 0.5 n_layer: 3
[TL] DynamicRNNLayer model/seq2seq/encode: n_hidden: 1024, in_dim: 3 in_shape: (32, ?, 1024) cell_fn: LSTMCell dropout: 0.5 n_layer: 3
[TL]        batch_size (concurrent processes): 32
[TL] DynamicRNNLayer model/seq2seq/decode: n_hidden: 1024, in_dim: 3 in_shape: (32, ?, 1024) cell_fn: LSTMCell dropout: 0.5 n_layer: 3
[TL]        batch_size (concurrent processes): 32
[TL] DenseLayer  model/output: 8004 No Activation
[TL]   param   0: model/embedding/seq_embedding/embeddings:0 (8004, 1024)       float32_ref

Epoch[1/50]:  10%|▉         | 271/2852 [01:48<16:58,  2.54it/s]